# Standardizing FBI Return A data

In [1]:
import pandas as pd
import struct
import sys
import glob

# Step 1: Parse data from fixed-width files

The code in this step is largely adopted from [Donald Braman's Return A conversion scripts](https://github.com/donaldbraman/fbi-reta-data) — specifically [schemagenerating.py](https://github.com/donaldbraman/fbi-reta-data/blob/master/script/schemagenerating.py) and [fwf2psv.py](https://github.com/donaldbraman/fbi-reta-data/blob/master/script/fwf2psv.py).

We also convert the FBI's textual representation of numbers into integers. (The raw data uses an unconventional system of denoting negative numbers.)

In [2]:
file_header_widths = [
    1, 2, 7, 2, 1, 2, 5, 2, 1, 7, 1,
    6, 4, 2, 1, 9, 3, 3, 9, 3, 3, 9,
    3, 3, 9, 9, 9, 1, 1, 1, 1, 24, 6,
    30, 30, 30, 30, 5, 1, 29
]

monthly_header_widths = [
    2, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1
] 

cards_0123_widths = [
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5
] 

card_4_widths = [
    3, 3, 7
]

mw =  monthly_header_widths + cards_0123_widths * 4 + card_4_widths
field_widths = file_header_widths + mw * 12
assert sum(field_widths) == 7385

In [3]:
file_header_names = [
    "id", "state", "agency_ori_7", "ori_group", "division", 
    "year", "sequence", "juv_age", "core_ci", "covered_by", "covered_by_group", 
    "last_update", "field_office", "num_months", "agency_count",
    "pop1", "county1", "msa1", 
    "pop2", "county2", "msa2", 
    "pop3", "county3", "msa3",
    "county1_pop", "county2_pop", "county3_pop", 
    "pop_source", 
    "follow_up", "special_mail_group", "special_mail_addr",
    "agency_name", "agency_state", 
    "address1", "address2", "address3", "address4", "zip",
    "old_pop_group",
    "unused_header"
]

monthly_header_names = [
    "month_in", "date_last_update", 
    "card_0_type", "card_1_type", "card_2_type", "card_3_type", "card_4_type", 
    "card_0_pt", "card_1_pt", "card_2_pt", "card_3_pt"
]

crime_card_prefixes = [
    "unfounded_",
    "actual_",
    "cleared_",
    "cleared_u18_"
]

cards_0123_names = [
    "murder",
    "manslaughter",
    "rape_total", "rape_by_force",  "rape_attempt",
    "robbery_total", "robbery_gun", "robbery_knife", "robbery_othweap",  "robbery_strong_arm",
    "assault_total", "assault_gun", "assault_knife", "assault_othweap", "assault_hands", "assault_simple", 
    "burglary_total", "burglary_forcible_entry", "burglary_no_forcible_entry", "burglary_attempt",
    "larceny", "motor_vehicle_theft", "auto_theft", "truck_bus_theft", "other_vehicle_theft", 
    "total_all_fields", "larceny_under_50_dollars", "unused"
]

cards =  [
    prefix + offense
        for prefix in crime_card_prefixes
              for offense in cards_0123_names
]

card_4_names = [
    "officers_killed_felony",
    "officers_killed_accident",
    "officers_assaulted"
]

card_names = monthly_header_names + cards + card_4_names

month_prefixes = [
    "jan_", "feb_", "mar_", "apr_", "may_", "jun_",
    "jul_", "aug_", "sep_", "oct_", "nov_", "dec_"
]

twelve_cards = [
    month + cn
        for month in month_prefixes
            for cn in card_names
]

field_names = file_header_names + twelve_cards

assert len(field_names) == len(field_widths)

In [4]:
cols_trimmed = [ c for c in field_names if (
    c in [ "year", "agency_ori_7", "agency_name", "ori_group", "pop1", "pop2", "pop3" ]
    or ((
        "_cleared_" in c
        or "_actual_" in c
    ) and ("murder" in c or "assault" in c) and ("18" not in c))
) ]

len(cols_trimmed)

175

In [5]:
cols_trimmed[:3]

['agency_ori_7', 'ori_group', 'year']

In [6]:
trimmed_indexes = [ field_names.index(c) for c in cols_trimmed ]
trimmed_indexes[:3]

[2, 3, 5]

In [7]:
# Make sure they're in correct order
assert (
    pd.Series(trimmed_indexes) - pd.Series(trimmed_indexes).shift(1)
).loc[lambda x: x < 0].pipe(len) == 0

In [8]:
def build_parser():
    total_width = sum(field_widths)
    fmtstring = " ".join("{}s".format(fw) for fw in field_widths)
    fieldstruct = struct.Struct(fmtstring)
    parse_bytes = fieldstruct.unpack_from
        
    def parse_line(line):
        line = line.strip().replace(b"\x00", b"")
        line = line + b" " * (total_width - len(line))
        return [ x.strip().decode("latin-1") for i, x in enumerate(parse_bytes(line))
            if i in trimmed_indexes ]

    return parse_line

parser = build_parser()

In [9]:
def convert_year(year_str):
    year_int = int(year_str)
    if year_int < 60: return 2000 + year_int
    else: return 1900 + year_int

In [10]:
NEGATIVE_SYMBOLS = "}JKLMNOPQR"

In [11]:
def parse_reta_num_string(num_string):
    try: return int(num_string)
    except:
        base = (-10 * int(num_string[:-1]))
        return base - NEGATIVE_SYMBOLS.index(num_string[-1])

Example:

In [12]:
parse_reta_num_string("00001J")

-11

In [13]:
def parse_file(path):
    sys.stderr.write(f"Parsing {path}\n")
    sys.stderr.flush()
    df = (
        pd.DataFrame(
            list(map(parser, open(path, "rb"))),
            columns = cols_trimmed
        )
        .replace("", pd.np.nan)
        .dropna(subset = [ "agency_ori_7", "year" ])
    )
    df["year"] = df["year"].apply(convert_year)
    
    for col in df.columns:
        if "pop" in col[:3]:
            df[col] = df[col].astype(int)
        elif "_actual_" in col or "_cleared_" in col:
            df[col] = df[col].apply(parse_reta_num_string).astype(int)
        else:
            pass
        
    return df

In [14]:
paths = sorted(glob.glob("../../data/raw/reta/*.DAT"))
len(paths)

58

In [15]:
parsed = pd.DataFrame()
for path in paths:
    parsed = pd.concat([ parsed, parse_file(path) ])

Parsing ../../data/raw/reta/RETA1960.DAT
Parsing ../../data/raw/reta/RETA1961.DAT
Parsing ../../data/raw/reta/RETA1962.DAT
Parsing ../../data/raw/reta/RETA1963.DAT
Parsing ../../data/raw/reta/RETA1964.DAT
Parsing ../../data/raw/reta/RETA1965.DAT
Parsing ../../data/raw/reta/RETA1966.DAT
Parsing ../../data/raw/reta/RETA1967.DAT
Parsing ../../data/raw/reta/RETA1968.DAT
Parsing ../../data/raw/reta/RETA1969.DAT
Parsing ../../data/raw/reta/RETA1970.DAT
Parsing ../../data/raw/reta/RETA1971.DAT
Parsing ../../data/raw/reta/RETA1972.DAT
Parsing ../../data/raw/reta/RETA1973.DAT
Parsing ../../data/raw/reta/RETA1974.DAT
Parsing ../../data/raw/reta/RETA1975.DAT
Parsing ../../data/raw/reta/RETA1976.DAT
Parsing ../../data/raw/reta/RETA1977.DAT
Parsing ../../data/raw/reta/RETA1978.DAT
Parsing ../../data/raw/reta/RETA1979.DAT
Parsing ../../data/raw/reta/RETA1980.DAT
Parsing ../../data/raw/reta/RETA1981.DAT
Parsing ../../data/raw/reta/RETA1982.DAT
Parsing ../../data/raw/reta/RETA1983.DAT
Parsing ../../da

In [16]:
parsed.head()

,agency_ori_7,ori_group,year,pop1,pop2,pop3,agency_name,jan_actual_murder,jan_actual_assault_total,jan_actual_assault_gun,...,dec_actual_assault_othweap,dec_actual_assault_hands,dec_actual_assault_simple,dec_cleared_murder,dec_cleared_assault_total,dec_cleared_assault_gun,dec_cleared_assault_knife,dec_cleared_assault_othweap,dec_cleared_assault_hands,dec_cleared_assault_simple
0,AL00100,9A,1960,165354,0,0,JEFFERSON,0,7,0,...,0,0,0,0,0,0,0,0,0,0
1,AL00101,4,1960,33054,0,0,BESSEMER,0,6,0,...,0,0,0,0,0,0,0,0,0,0
2,AL00102,1C,1960,340887,0,0,BIRMINGHAM,2,82,0,...,0,0,0,0,0,0,0,0,0,0
3,AL00103,5,1960,12680,0,0,MOUNTAIN BROOK,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AL00104,5,1960,15816,0,0,FAIRFIELD,0,4,0,...,0,0,0,0,0,0,0,0,0,0


# Step 2: Fix apparent ORI errors

In this step, we fix a few rows where an agency's ORI code appears to be mislabeled, based on duplicate entries for the same ORI.

In [17]:
def fix_errors(df):

    df.loc[(
        (df["agency_ori_7"] == "VA02101") &
        (df["year"].isin([ 1967, 1970 ])) &
        (df["ori_group"] == "9D")
    ), "agency_ori_7"] = "VA021SP"

    df.loc[(
        (df["agency_ori_7"] == "SC02901") &
        (df["year"] == 1972) &
        (df["ori_group"] == "9A")
    ), "agency_ori_7"] = "VA02901"

    return df

In [18]:
parsed = fix_errors(parsed)

In [19]:
parsed.duplicated(subset = [ "year", "agency_ori_7" ]).sum()

1

# Step 3: Extract and save agency-year metadata

In [20]:
agency_metadata = (
    parsed
    [[
        "year",
        "agency_ori_7",
        "agency_name",
        "ori_group",
        "pop1",
        "pop2",
        "pop3"
    ]]
    .drop_duplicates()
    .astype({
        "pop1": int,
        "pop2": int,
        "pop3": int
    })
)

agency_metadata.head()

,year,agency_ori_7,agency_name,ori_group,pop1,pop2,pop3
0,1960,AL00100,JEFFERSON,9A,165354,0,0
1,1960,AL00101,BESSEMER,4,33054,0,0
2,1960,AL00102,BIRMINGHAM,1C,340887,0,0
3,1960,AL00103,MOUNTAIN BROOK,5,12680,0,0
4,1960,AL00104,FAIRFIELD,5,15816,0,0


In [21]:
(
    agency_metadata
    .to_csv(
        "../../data/reta/standardized/reta-agency-metadata.csv",
        index = False
    )
)

# Step 4: Summarize into annual counts

Note: In this step, we convert the "wide" structure of the data into an easier-to-analyze structure.

In [22]:
tidy_counts = (
    parsed
    .melt(
        id_vars = [ "agency_ori_7", "year" ],
        value_vars = [ c for c in parsed.columns
            if c.count("_") > 0 and c.split("_")[1] in [ "actual", "cleared" ] ]
    )

    .assign(
        offense = lambda x: x["variable"].apply(lambda s: s.split("_", 2)[-1]),
        
        count_type = lambda x: x["variable"].apply(lambda s: s.split("_", 2)[-2]),
    )

)

tidy_counts.head()

,agency_ori_7,year,variable,value,offense,count_type
0,AL00100,1960,jan_actual_murder,0,murder,actual
1,AL00101,1960,jan_actual_murder,0,murder,actual
2,AL00102,1960,jan_actual_murder,2,murder,actual
3,AL00103,1960,jan_actual_murder,0,murder,actual
4,AL00104,1960,jan_actual_murder,0,murder,actual


In [23]:
tidy_counts["count_type"].value_counts().sort_index()

actual     80555580
cleared    80555580
Name: count_type, dtype: int64

In [24]:
tidy_counts["offense"].value_counts().sort_index()

assault_gun        23015880
assault_hands      23015880
assault_knife      23015880
assault_othweap    23015880
assault_simple     23015880
assault_total      23015880
murder             23015880
Name: offense, dtype: int64

In [25]:
# Overview of negative numbers in the data
(
    tidy_counts
    .loc[lambda df: df["value"] < 0]
    ["value"]
    .value_counts()
    .sort_index(ascending = False)
)

-1      5275
-2       498
-3       136
-4        58
-5        44
-6        22
-7        23
-8        12
-9         9
-10        6
-11        9
-12        4
-13        5
-14        4
-15        4
-16        6
-17        4
-18        4
-19        4
-20        2
-21       10
-23        1
-25        2
-26        1
-31        2
-33        3
-34        1
-36        1
-38        1
-39        1
-42        2
-43        2
-44        1
-55        2
-60        1
-66        1
-67        1
-69        2
-74        1
-79        1
-85        2
-89        1
-106       1
-128       1
-142       1
-174       1
-619       1
-629       1
Name: value, dtype: int64

In [26]:
annual_counts = (
    tidy_counts
    .groupby([
        "agency_ori_7",
        "year",
        "offense",
        "count_type"
    ])
    ["value"]
    .sum()
    .unstack()
    .reset_index()
)

annual_counts.head()

count_type,agency_ori_7,year,offense,actual,cleared
0,AK00101,1960,assault_gun,0,0
1,AK00101,1960,assault_hands,0,0
2,AK00101,1960,assault_knife,0,0
3,AK00101,1960,assault_othweap,0,0
4,AK00101,1960,assault_simple,0,0


# Step 5: Save counts for subsequent analysis

In [27]:
(
    annual_counts
    .to_csv(
        "../../data/standardized/reta-annual-counts.csv",
        index = False
    )
)

---

---

---